# Start

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/NLP/FINAL
#%cd /content/drive/My\ Drive/New

/content/drive/My Drive/NLP/FINAL


# Model

## Preprocessing

In [ ]:
import numpy as np
import torchtext
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import pickle
import string
from tqdm.notebook import tqdm
import time
import pandas as pd

In [ ]:
class Params():
    def __init__(self):
        self.num_vocab = 400001
        self.num_pos = 17
        self.num_dep = 44
        self.num_ent = 66

        self.dim_word = 200
        self.dim_pos = 25
        self.dim_dep = 25
        self.dim_ent = 25

        self.dim_hidden = 100

        self.num_class = 10
        self.batch_size = 20

        self.learn_rate = 1e-3
        self.learn_rate_decay = 0.92
        self.step_size = 50
        self.weight_decay = 1e-5
        self.dropout = 0.0
        self.alpha = 1
        self.total_epoch = 5
        self.max_norm = 10

params = Params()

In [ ]:
glove = torchtext.vocab.GloVe(name='6B', dim=200)
glove.itos.append('UKN')
glove.stoi['UKN'] = params.num_vocab - 1

In [ ]:
pos_dict = {'ADJ': 0, 'ADP': 1, 'ADV': 2, 'AUX': 3, 'CCONJ': 4, 'DET': 5, 'INTJ': 6, 'NOUN': 7, 'NUM': 8, 
            'PART': 9, 'PRON': 10, 'PROPN': 11, 'PUNCT': 12, 'SCONJ': 13, 'SYM': 14, 'VERB': 15, 'X': 16}

In [ ]:
dep_dict = {'acl': 0, 'acl:relcl': 1, 'advcl': 2, 'advmod': 3, 'amod': 4, 'appos': 5, 'aux': 6, 'aux:pass': 7, 'case': 8, 
            'cc': 9, 'cc:preconj': 10, 'ccomp': 11, 'compound': 12, 'compound:prt': 13, 'conj': 14, 'cop': 15, 'csubj': 16, 
            'csubj:pass': 17, 'det': 18, 'det:predet': 19, 'discourse': 20, 'expl': 21, 'fixed': 22, 'flat': 23, 'goeswith': 24, 
            'iobj': 25, 'list': 26, 'mark': 27, 'nmod': 28, 'nmod:npmod': 29, 'nmod:poss': 30, 'nmod:tmod': 31, 'nsubj': 32, 
            'nsubj:pass': 33, 'nummod': 34, 'obj': 35, 'obl': 36, 'obl:npmod': 37, 'obl:tmod': 38, 'parataxis': 39, 'punct': 40, 
            'root': 41, 'vocative': 42, 'xcomp': 43}

In [ ]:
ent_dict = {'B-CARDINAL': 0, 'B-DATE': 1, 'B-EVENT': 2, 'B-FAC': 3, 'B-GPE': 4, 'B-LAW': 5, 'B-LOC': 6, 'B-MONEY': 7, 'B-NORP': 8, 
            'B-ORG': 9, 'B-PERCENT': 10, 'B-PERSON': 11, 'B-PRODUCT': 12,'B-QUANTITY': 13, 'B-TIME': 14, 'B-WORK_OF_ART': 15, 
            'I-CARDINAL': 16, 'I-DATE': 17, 'I-EVENT': 18, 'I-FAC': 19, 'I-GPE': 20, 'I-LAW': 21, 'I-LOC': 22, 'I-MONEY': 23, 
            'I-NORP': 24, 'I-ORG': 25, 'I-PERCENT': 26,'I-PERSON': 27, 'I-PRODUCT': 28, 'I-QUANTITY': 29, 'I-TIME': 30, 'I-WORK_OF_ART': 31, 
            'L-CARDINAL': 32, 'L-DATE': 33, 'L-EVENT': 34, 'L-FAC': 35, 'L-GPE': 36, 'L-LAW': 37, 'L-LOC': 38, 'L-MONEY': 39, 'L-NORP': 40, 
            'L-ORG': 41, 'L-PERCENT': 42, 'L-PERSON': 43, 'L-PRODUCT': 44, 'L-QUANTITY': 45,'L-TIME': 46, 'L-WORK_OF_ART': 47, 'O': 48, 
            'U-CARDINAL': 49, 'U-DATE': 50, 'U-EVENT': 51, 'U-FAC': 52, 'U-GPE': 53, 'U-LANGUAGE': 54, 'U-LAW': 55, 'U-LOC': 56, 'U-MONEY': 57, 
            'U-NORP': 58, 'U-ORDINAL': 59, 'U-ORG': 60, 'U-PERSON': 61, 'U-PRODUCT': 62, 'U-QUANTITY': 63, 'U-TIME': 64, 'U-WORK_OF_ART': 65}

In [ ]:
class Data(object):
    def __init__(self, words_seq, pos_seq, deps_seq, ents_seq, indx_path1, indx_path2, dep_path1, dep_path2, childs_path1, childs_path2, rels, rels1, rels2, 
                 len_path1, len_path2, num_child_path1, num_child_path2, words_seq_id, pos_seq_id, deps_seq_id, dep_path1_id, dep_path2_id, ents_seq_id):
        self.words_seq = words_seq
        self.pos_seq = pos_seq
        self.deps_seq = deps_seq
        self.ents_seq = ents_seq
        self.indx_path1 = indx_path1
        self.indx_path2 = indx_path2
        self.dep_path1 = dep_path1
        self.dep_path2 = dep_path2
        self.childs_path1 = childs_path1
        self.childs_path2 = childs_path2
        self.rels = rels
        self.rels1 = rels1
        self.rels2 = rels2
        self.len_path1 = len_path1
        self.len_path2 = len_path2
        self.num_child_path1 = num_child_path1
        self.num_child_path2 = num_child_path2
        self.words_seq_id = words_seq_id
        self.pos_seq_id = pos_seq_id
        self.deps_seq_id = deps_seq_id
        self.dep_path1_id = dep_path1_id
        self.dep_path2_id = dep_path2_id
        self.ents_seq_id = ents_seq_id
        
        self.size = len(words_seq)
        self.offset = 0
    
    def next_batch(self, batch_size):
        if self.offset+batch_size > self.size:
            self.offset = self.offset+batch_size-self.size
        batch = Data(self.words_seq[self.offset:self.offset+batch_size], 
                     self.pos_seq[self.offset:self.offset+batch_size], 
                     self.deps_seq[self.offset:self.offset+batch_size], 
                     self.ents_seq[self.offset:self.offset+batch_size], 
                     self.indx_path1[self.offset:self.offset+batch_size], 
                     self.indx_path2[self.offset:self.offset+batch_size], 
                     self.dep_path1[self.offset:self.offset+batch_size], 
                     self.dep_path2[self.offset:self.offset+batch_size], 
                     self.childs_path1[self.offset:self.offset+batch_size], 
                     self.childs_path2[self.offset:self.offset+batch_size], 
                     self.rels[self.offset:self.offset+batch_size], 
                     self.rels1[self.offset:self.offset+batch_size], 
                     self.rels2[self.offset:self.offset+batch_size], 
                     self.len_path1[self.offset:self.offset+batch_size], 
                     self.len_path2[self.offset:self.offset+batch_size], 
                     self.num_child_path1[self.offset:self.offset+batch_size], 
                     self.num_child_path2[self.offset:self.offset+batch_size], 
                     self.words_seq_id[self.offset:self.offset+batch_size], 
                     self.pos_seq_id[self.offset:self.offset+batch_size], 
                     self.deps_seq_id[self.offset:self.offset+batch_size], 
                     self.dep_path1_id[self.offset:self.offset+batch_size], 
                     self.dep_path2_id[self.offset:self.offset+batch_size], 
                     self.ents_seq_id[self.offset:self.offset+batch_size])
        self.offset += batch_size
        return batch

In [ ]:
def prepare_input(file_path):
    f = open(file_path, 'rb')
    words_seq, pos_seq, deps_seq, ents_seq, indx_path1, indx_path2, dep_path1, dep_path2, childs_path1, childs_path2, rels = pickle.load(f)
    f.close()

    num_data = len(words_seq)

    # replace unknown words
    for i in range(num_data):
        for j, word in enumerate(words_seq[i]):
            if word not in glove.itos:
                words_seq[i][j] = 'UKN'

    # length of LCA paths
    len_path1 = [len(indx_path1[i]) for i in range(num_data)]
    len_path2 = [len(indx_path2[i]) for i in range(num_data)]
    
    # number of children of words in LCA paths
    num_child_path1 = [[len(childs) for childs in childs_path1[i]] for i in range(num_data)]
    num_child_path2 = [[len(childs) for childs in childs_path2[i]] for i in range(num_data)]
    
    # word -> id
    words_seq_id = [[glove.stoi[word] for word in words_seq[i]] for i in range(num_data)]

    # pos -> id
    pos_seq_id = [[pos_dict[pos] for pos in pos_seq[i]] for i in range(num_data)]

    # dep -> id
    deps_seq_id = [[dep_dict[dep] for dep in deps_seq[i]] for i in range(num_data)]
    dep_path1_id = [[dep_dict[dep] for dep in dep_path1[i]] for i in range(num_data)]
    dep_path2_id = [[dep_dict[dep] for dep in dep_path2[i]] for i in range(num_data)]

    # ent -> id
    ents_seq_id = [[ent_dict[ent] for ent in ents_seq[i]] for i in range(num_data)]

    # rels for v15+
    rels1 = []
    rels2 = []
    for i in range(num_data):
        if rels[i] % 2 == 0:
            rels1.append(int(rels[i] / 2))
            rels2.append(9)
        else:
            rels1.append(9)
            rels2.append(int(rels[i] / 2))

    return Data(words_seq, pos_seq, deps_seq, ents_seq, indx_path1, indx_path2, dep_path1, dep_path2, childs_path1, childs_path2, rels, rels1, rels2,
                len_path1, len_path2, num_child_path1, num_child_path2, words_seq_id, pos_seq_id, deps_seq_id, dep_path1_id, dep_path2_id, ents_seq_id)

## BiLSTM+Tree-LSTM

In [ ]:
class BiLSTM_TreeLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM_TreeLSTM, self).__init__()
        glove = torchtext.vocab.GloVe(name='6B', dim=200)
        glove.itos.append('UKN')
        glove.stoi['UKN'] = params.num_vocab - 1

        ## EMBEDDING LAYER ##
        self.UKN_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(1, params.dim_word))).cuda()
        self.word_embedding = torch.cat([glove.vectors.cuda(), self.UKN_embedding], 0)
        self.pos_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(params.num_pos, params.dim_pos)))
        self.dep_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(params.num_dep, params.dim_dep)))
        self.ent_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(params.num_ent, params.dim_ent)))

        ## SEQUENCE LAYER ##
        self.seq_lstm = nn.LSTM(params.dim_word+params.dim_pos, params.dim_hidden, bidirectional=True)
        self.seq_h0 = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(2, 1, params.dim_hidden)))
        self.seq_c0 = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(2, 1, params.dim_hidden)))

        ## ENTITY DETECTION ##
        self.dect_cell = nn.RNNCell(params.dim_hidden*2+params.dim_ent, params.dim_hidden, bias=True, nonlinearity='tanh')
        self.dect_ent0 = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(1, params.dim_ent)))
        self.dect_Wy = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(params.dim_hidden, params.num_ent)))
        self.dect_by = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(1, params.num_ent)))

        ## DEPENDENCY LAYER ##
        self.dep_Wi = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden*2+params.dim_dep+params.dim_ent, params.dim_hidden)))
        self.dep_Ui = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden, params.dim_hidden)))
        self.dep_Uit = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(1, params.dim_hidden, params.dim_hidden)))
        self.dep_bi = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, 1, params.dim_hidden)))

        self.dep_Wf = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden*2+params.dim_dep+params.dim_ent, params.dim_hidden)))
        self.dep_Uf = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden, params.dim_hidden)))
        self.dep_Uft = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden, params.dim_hidden)))
        self.dep_bf = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, 1, params.dim_hidden)))

        self.dep_Wo = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden*2+params.dim_dep+params.dim_ent, params.dim_hidden)))
        self.dep_Uo = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden, params.dim_hidden)))
        self.dep_Uot = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(1, params.dim_hidden, params.dim_hidden)))
        self.dep_bo = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, 1, params.dim_hidden)))

        self.dep_Wu = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden*2+params.dim_dep+params.dim_ent, params.dim_hidden)))
        self.dep_Uu = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, params.dim_hidden, params.dim_hidden)))
        self.dep_Uut = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(1, params.dim_hidden, params.dim_hidden)))
        self.dep_bu = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, 1, params.dim_hidden)))

        self.dep_c0 = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, 1, params.dim_hidden)))
        self.dep_h0 = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(3, 1, params.dim_hidden)))    

        ## RELATION CLASSIFICATION ##
        self.rc = nn.Sequential(
            nn.Linear(params.dim_hidden*5, params.dim_hidden),
            nn.Tanh(),
            nn.Linear(params.dim_hidden, params.num_class)
        )

        # dropout layer
        self.dropout = nn.Dropout(p=params.dropout)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()


    def forward(self, Data):
        # result of entity dectection
        dect_y = []

        # result of dependency layer
        dep_y = torch.tensor([]).cuda()
        dep_y_ = torch.tensor([]).cuda()


        ## EMBEDDING LAYER ##
        word_embd = [F.embedding(torch.tensor(Data.words_seq_id[i]).cuda(), self.word_embedding) for i in range(Data.size)]
        pos_embd = [F.embedding(torch.tensor(Data.pos_seq_id[i]).cuda(), self.pos_embedding) for i in range(Data.size)]
        dep_embd = [F.embedding(torch.tensor(Data.deps_seq_id[i]).cuda(), self.dep_embedding) for i in range(Data.size)]
        dep1_embd = [F.embedding(torch.tensor(Data.dep_path1_id[i]).cuda(), self.dep_embedding) for i in range(Data.size)]
        dep2_embd = [F.embedding(torch.tensor(Data.dep_path2_id[i]).cuda(), self.dep_embedding) for i in range(Data.size)]
        ent_embd = [F.embedding(torch.tensor(Data.ents_seq_id[i]).cuda(), self.ent_embedding) for i in range(Data.size)]
        ent1_embd = [F.embedding(torch.tensor(Data.dep_path1_id[i]).cuda(), self.ent_embedding) for i in range(Data.size)]
        ent2_embd = [F.embedding(torch.tensor(Data.dep_path2_id[i]).cuda(), self.ent_embedding) for i in range(Data.size)]


        for i in range(Data.size):
            seq_x = torch.cat([word_embd[i], pos_embd[i]], 1)
            seq_x = torch.unsqueeze(seq_x, 1)

            ## SEQUENCE LAYER ##
            seq_h, (_, _) = self.seq_lstm(seq_x, (self.seq_h0, self.seq_c0))
            seq_h_ = torch.squeeze(seq_h, 1)


            ## ENTITY DECTECTION ##
            dect_ent = self.dect_ent0
            dect_y_ = torch.tensor([]).cuda()
            dect_y_id = []

            for j in range(len(Data.words_seq[i])):
                dect_xt = torch.cat([seq_h[j], dect_ent], 1)
                dect_ht = self.dect_cell(dect_xt)
                dect_yt = torch.mm(dect_ht, self.dect_Wy) + self.dect_by
                dect_yt_id = torch.argmax(dect_yt)

                dect_y_ = torch.cat([dect_y_, dect_yt], 0)
                dect_y_id.append(dect_yt_id)

                dect_ent = torch.unsqueeze(self.ent_embedding[dect_yt_id], 0)

            dect_y.append(dect_y_)

            dect_ent_embd = F.embedding(torch.tensor(dect_y_id).cuda(), self.ent_embedding)


            ## STACKING LAYER ##
            dep_embd_ = self.dropout(dep_embd[i])
            #ent_embd_ = self.dropout(dect_ent_embd)
            ent_embd_ = self.dropout(ent_embd[i])
            dep_x = torch.unsqueeze(torch.cat([seq_h_, dep_embd_, ent_embd_], 1), 1)


            ## DEPENDENCY LAYER ##
            # bottom-up

            def lstmcell(dep_x, k): # bottom-up
                it = self.sigmoid(torch.mm(dep_x[k], self.dep_Wi[0]) + torch.mm(self.dep_h0[0], self.dep_Ui[0]) + self.dep_bi[0])
                ot = self.sigmoid(torch.mm(dep_x[k], self.dep_Wo[0]) + torch.mm(self.dep_h0[0], self.dep_Uo[0]) + self.dep_bo[0])
                ut = self.tanh(torch.mm(dep_x[k], self.dep_Wu[0]) + torch.mm(self.dep_h0[0], self.dep_Uu[0]) + self.dep_bu[0])
                ft = self.sigmoid(torch.mm(dep_x[k], self.dep_Wf[0]) + torch.mm(self.dep_h0[0], self.dep_Uf[0]) + self.dep_bf[0])
                c = torch.mul(it, ut) + torch.mul(ft, self.dep_c0[0])
                h = torch.mul(ot, self.tanh(c))
                return c, h

            def child_sum_lstm(dep_x, indx, prev_indx1, prev_indx2, childs, dep_ct, dep_ht):
                it = torch.mm(dep_x[indx], self.dep_Wi[0]) + self.dep_bi[0]
                ot = torch.mm(dep_x[indx], self.dep_Wo[0]) + self.dep_bo[0]
                ut = torch.mm(dep_x[indx], self.dep_Wu[0]) + self.dep_bu[0]


                for k in childs:
                    if k==prev_indx1:
                        it += torch.mm(dep_ht[0], self.dep_Ui[0])
                        ot += torch.mm(dep_ht[0], self.dep_Uo[0])
                        ut += torch.mm(dep_ht[0], self.dep_Uu[0])
                    elif k==prev_indx2:
                        it += torch.mm(dep_ht[1], self.dep_Ui[0])
                        ot += torch.mm(dep_ht[1], self.dep_Uo[0])
                        ut += torch.mm(dep_ht[1], self.dep_Uu[0])
                
                it = self.sigmoid(it)
                ot = self.sigmoid(ot)
                ut = self.tanh(ut)
                
                ct = torch.mul(it, ut)

                for k in childs:
                    ftk = torch.mm(dep_x[indx], self.dep_Wf[0]) + self.dep_bf[0]
                    if k==prev_indx1:
                        ftk += torch.mm(dep_ht[0], self.dep_Uf[0])
                        ct += torch.mul(self.sigmoid(ftk), dep_ct[0])
                    elif k==prev_indx2:
                        ftk += (torch.mm(dep_ht[0], self.dep_Uf[0]) + torch.mm(dep_ht[1], self.dep_Uf[0]))
                        ct += torch.mul(self.sigmoid(ftk), dep_ct[1])
                
                ht = torch.mul(ot, self.tanh(ct))
                return ct, ht
            

            dep_ct1 = self.dep_c0[0]
            dep_ht1 = self.dep_h0[0]
            for j in range(Data.len_path1[i]-1):
                dep_ct1, dep_ht1 = child_sum_lstm(dep_x, Data.indx_path1[i][j], Data.indx_path1[i][j-1], -1, 
                                                  Data.childs_path1[i][j], [dep_ct1], [dep_ht1])
            
            dep_ct2 = self.dep_c0[0]
            dep_ht2 = self.dep_h0[0]
            for j in range(Data.len_path2[i]-1):
                dep_ct2, dep_ht2 = child_sum_lstm(dep_x, Data.indx_path2[i][j], Data.indx_path2[i][j-1], -1, 
                                                  Data.childs_path2[i][j], [dep_ct2], [dep_ht2])
            
            if len(Data.indx_path1[i])==1:
                prev_indx1 = -1
            else:
                prev_indx1 = Data.indx_path1[i][-2]
            if len(Data.indx_path2[i])==1:
                prev_indx2 = -1
            else:
                prev_indx2 = Data.indx_path2[i][-2]
            
            dep_c_bu, dep_h_bu = child_sum_lstm(dep_x, Data.indx_path1[i][-1], prev_indx1, prev_indx2, 
                                                Data.childs_path1[i][-1], [dep_ct1, dep_ct2], [dep_ht1, dep_ht2])
            
            tmp1 = torch.unsqueeze(seq_h_[Data.indx_path1[i][0]][:params.dim_hidden] + seq_h_[Data.indx_path1[i][0]][params.dim_hidden:], 0)
            tmp2 = torch.unsqueeze(seq_h_[Data.indx_path2[i][0]][:params.dim_hidden] + seq_h_[Data.indx_path2[i][0]][params.dim_hidden:], 0)

            # top-down
            dep_ct3 = self.dep_c0[1]
            dep_ht3 = self.dep_h0[1]
            for j in range(Data.len_path1[i]-1, -1, -1):
                it = self.sigmoid(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wi[1]) + torch.mm(dep_ht3, self.dep_Ui[1]) + self.dep_bi[1])
                ft = self.sigmoid(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wf[1]) + torch.mm(dep_ht3, self.dep_Uf[1]) + self.dep_bf[1])
                ot = self.sigmoid(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wo[1]) + torch.mm(dep_ht3, self.dep_Uo[1]) + self.dep_bo[1])
                ut = self.tanh(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wu[1]) + torch.mm(dep_ht3, self.dep_Uu[1]) + self.dep_bu[1])
                dep_ct3 = torch.mul(it, ut) + torch.mul(ft, dep_ct3)
                dep_ht3 = torch.mul(ot, self.tanh(dep_ct3))
            
            dep_ct4 = self.dep_c0[2]
            dep_ht4 = self.dep_h0[2]
            for j in range(Data.len_path2[i]-1, -1, -1):
                it = self.sigmoid(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wi[2]) + torch.mm(dep_ht4, self.dep_Ui[2]) + self.dep_bi[2])
                ft = self.sigmoid(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wf[2]) + torch.mm(dep_ht4, self.dep_Uf[2]) + self.dep_bf[2])
                ot = self.sigmoid(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wo[2]) + torch.mm(dep_ht4, self.dep_Uo[2]) + self.dep_bo[2])
                ut = self.tanh(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wu[2]) + torch.mm(dep_ht4, self.dep_Uu[2]) + self.dep_bu[2])
                dep_ct4 = torch.mul(it, ut) + torch.mul(ft, dep_ct4)
                dep_ht4 = torch.mul(ot, self.tanh(dep_ct4))
            
            dep_y = torch.cat([dep_y, torch.cat([dep_h_bu, dep_ht3, dep_ht4, tmp1, tmp2], 1)], 0)

            dep_ct5 = self.dep_c0[1]
            dep_ht5 = self.dep_h0[1]
            for j in range(Data.len_path2[i]-1, -1, -1):
                it = self.sigmoid(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wi[1]) + torch.mm(dep_ht5, self.dep_Ui[1]) + self.dep_bi[1])
                ft = self.sigmoid(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wf[1]) + torch.mm(dep_ht5, self.dep_Uf[1]) + self.dep_bf[1])
                ot = self.sigmoid(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wo[1]) + torch.mm(dep_ht5, self.dep_Uo[1]) + self.dep_bo[1])
                ut = self.tanh(torch.mm(dep_x[Data.indx_path2[i][j]], self.dep_Wu[1]) + torch.mm(dep_ht5, self.dep_Uu[1]) + self.dep_bu[1])
                dep_ct5 = torch.mul(it, ut) + torch.mul(ft, dep_ct5)
                dep_ht5 = torch.mul(ot, self.tanh(dep_ct5))
            
            dep_ct6 = self.dep_h0[2]
            dep_ht6 = self.dep_h0[2]
            for j in range(Data.len_path1[i]-1, -1, -1):
                it = self.sigmoid(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wi[2]) + torch.mm(dep_ht6, self.dep_Ui[2]) + self.dep_bi[2])
                ft = self.sigmoid(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wf[2]) + torch.mm(dep_ht6, self.dep_Uf[2]) + self.dep_bf[2])
                ot = self.sigmoid(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wo[2]) + torch.mm(dep_ht6, self.dep_Uo[2]) + self.dep_bo[2])
                ut = self.tanh(torch.mm(dep_x[Data.indx_path1[i][j]], self.dep_Wu[2]) + torch.mm(dep_ht6, self.dep_Uu[2]) + self.dep_bu[2])
                dep_ct6 = torch.mul(it, ut) + torch.mul(ft, dep_ct6)
                dep_ht6 = torch.mul(ot, self.tanh(dep_ct6))
            
            dep_y_ = torch.cat([dep_y_, torch.cat([dep_h_bu, dep_ht5, dep_ht6, tmp2, tmp1], 1)], 0)
        

        ## RELATION CLASSIFICATION ##
        rc_y = self.rc(dep_y)
        rc_y_ = self.rc(dep_y_)

        return dect_y, rc_y, rc_y_

## BiLSTM

In [ ]:
class BiLSTM_TreeLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM_TreeLSTM, self).__init__()
        glove = torchtext.vocab.GloVe(name='6B', dim=200)
        glove.itos.append('UKN')
        glove.stoi['UKN'] = params.num_vocab - 1

        ## EMBEDDING LAYER ##
        self.UKN_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(1, params.dim_word))).cuda()
        self.word_embedding = torch.cat([glove.vectors.cuda(), self.UKN_embedding], 0)
        self.pos_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(params.num_pos, params.dim_pos)))
        self.dep_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(params.num_dep, params.dim_dep)))
        self.ent_embedding = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(params.num_ent, params.dim_ent)))

        ## SEQUENCE LAYER ##
        self.seq_lstm = nn.LSTM(params.dim_word+params.dim_pos+params.dim_ent+params.dim_dep, params.dim_hidden, bidirectional=True)
        self.seq_h0 = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(2, 1, params.dim_hidden)))
        self.seq_c0 = nn.Parameter(nn.init.xavier_normal_(torch.Tensor(2, 1, params.dim_hidden)))

        ## RELATION CLASSIFICATION ##
        self.rc = nn.Sequential(
            nn.Linear(params.dim_hidden, params.dim_hidden),
            nn.Tanh(),
            nn.Linear(params.dim_hidden, params.num_class)
        )

        # dropout layer
        self.dropout = nn.Dropout(p=params.dropout)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()


    def forward(self, Data):
        ## EMBEDDING LAYER ##
        word_embd = [F.embedding(torch.tensor(Data.words_seq_id[i]).cuda(), self.word_embedding) for i in range(Data.size)]
        pos_embd = [F.embedding(torch.tensor(Data.pos_seq_id[i]).cuda(), self.pos_embedding) for i in range(Data.size)]
        dep_embd = [F.embedding(torch.tensor(Data.deps_seq_id[i]).cuda(), self.dep_embedding) for i in range(Data.size)]
        ent_embd = [F.embedding(torch.tensor(Data.ents_seq_id[i]).cuda(), self.ent_embedding) for i in range(Data.size)]


        for i in range(Data.size):
            seq_x = torch.cat([word_embd[i], pos_embd[i], ent_embd[i], dep_embd[i]], 1)
            seq_x = torch.unsqueeze(seq_x, 1)

            ## SEQUENCE LAYER ##
            seq_h, (_, _) = self.seq_lstm(seq_x, (self.seq_h0, self.seq_c0))
            seq_h_ = torch.squeeze(seq_h, 1)
            h, _ = torch.max(seq_h[:, :, 0:params.dim_hidden]+seq_h[:, :, params.dim_hidden:], 0)
            h = torch.nn.functional.dropout(h, 0.3)

        

        ## RELATION CLASSIFICATION ##
        rc_y = self.rc(dep_y)
        rc_y_ = self.rc(dep_y_)

        return dect_y, rc_y, rc_y_

## Other

In [ ]:
# calculate weight
rels = []
ents = []

f = open('data/train_path', 'rb')
_, _, _, ents_seq1, _, _, _, _, _, _, rels1 = pickle.load(f)
f.close()
rels.extend(rels1)
for ent in ents_seq1:
    ents.extend(ent)

f = open('data/test_path', 'rb')
_, _, _, ents_seq2, _, _, _, _, _, _, rels2 = pickle.load(f)
f.close()
rels.extend(rels2)
for ent in ents_seq2:
    ents.extend(ent)

for i in range(10717):
    rels[i] = int(rels[i] / 2)

rel_freq = pd.value_counts(rels).sort_index().to_list()
rel_weight = [1e3 / rel_freq[i] for i in range(params.num_class)]
ent_freq = pd.value_counts(ents).sort_index().to_list()
ent_weight = [1e3 / ent_freq[i] for i in range(params.num_ent)]

In [ ]:
criterion1 = nn.CrossEntropyLoss(weight=torch.tensor(ent_weight, dtype=torch.float))
criterion2 = nn.CrossEntropyLoss(weight=torch.tensor(rel_weight, dtype=torch.float))

In [ ]:
#model = BiLSTM_TreeLSTM()
#model = model.cuda()
#optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=False)
#scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
#criterion1 = nn.CrossEntropyLoss(weight=torch.tensor(ent_weight, dtype=torch.float))
#criterion2 = nn.CrossEntropyLoss(weight=torch.tensor(rel_weight, dtype=torch.float))
#start_epoch = 1

In [ ]:
# load model
#checkpoint = torch.load('Model/treelstm_v15.pth')
#model.load_state_dict(checkpoint['net'])
#model = model.cuda()
#optimizer.load_state_dict(checkpoint['optimizer'])
#start_epoch = checkpoint['epoch'] + 1

# Train

In [ ]:
train_data = prepare_input('data/train_path')
test_data = prepare_input('data/test_path')

In [ ]:
def eval(epoch):
    model.eval()
    loss_dect = 0
    loss_dep = 0
    count = 0
    cond_mat = torch.zeros(19, 19)
    for _ in range(90):
        batch_data = test_data.next_batch(30)
        dect_y, rc_y, rc_y_= model(batch_data)

        torch.cat([rc_y, rc_y_], 1)
        
        for i in range(batch_data.size):
            loss_dect = loss_dep + criterion1(dect_y[i].cpu(), torch.tensor(batch_data.ents_seq_id[i])).item()
    
        loss_dep = loss_dep + criterion2(rc_y.cpu(), torch.tensor(batch_data.rels1)).item() + criterion2(rc_y_.cpu(), torch.tensor(batch_data.rels2)).item()

        for i in range(batch_data.size):
            if torch.argmax(rc_y[i])==9 and torch.argmax(rc_y_[i])==9:
                rel_pred = 18
            else:
                rel1 = torch.max(rc_y[i][0:9])
                rel2 = torch.max(rc_y_[i][0:9])
                if rel1 > rel2:
                    rel_pred = 2*torch.argmax(rc_y[i][0:9]).item()
                else:
                    rel_pred = 2*torch.argmax(rc_y_[i][0:9]).item() + 1

            cond_mat[rel_pred, batch_data.rels[i]] += 1
            if int(rel_pred/2)==int(batch_data.rels[i]/2):
                count += 1
    
    TP = (torch.tensor([cond_mat[i, i] for i in range(19)]))
    FP = torch.sum(cond_mat, 1) - TP
    FN = torch.sum(cond_mat, 0) - TP
    P = TP / (TP + FP + 1e-5)
    R = TP / (TP + FN + 1e-5)
    tmp = 2 / (1/P+1/R)
    P_ = torch.mean(torch.cat([P[0:5], P[6:18]], 0))
    R_ = torch.mean(torch.cat([R[0:5], R[6:18]], 0))
    f1 = torch.mean(torch.cat([tmp[0:5], tmp[6:18]], 0))
        
    print("epoch: %d, accuracy:  %2.2f%%, loss_dect: %2.4f, loss_dep: %2.4f, loss_total: %2.4f, R: %2.2f%%, P: %2.2f%%, f1_macro: %2.2f%%" % 
          (epoch, torch.trace(cond_mat)/27, loss_dect/90, loss_dep/90, (loss_dect+loss_dep)/90, R_*100, P_*100, f1*100))
    time.sleep(0.1)

In [ ]:
def train():
    model.train()
    batch_data = train_data.next_batch(params.batch_size)
    
    optimizer.zero_grad()

    dect_y, rc_y, rc_y_= model(batch_data)

    loss_dect = 0
    for i in range(batch_data.size):
        loss_dect = loss_dect + criterion1(dect_y[i].cpu(), torch.tensor(batch_data.ents_seq_id[i]))
    
    loss_dep = criterion2(rc_y.cpu(), torch.tensor(batch_data.rels1)) + criterion2(rc_y_.cpu(), torch.tensor(batch_data.rels2))
    
    loss = loss_dect * params.alpha + loss_dep
    
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=params.max_norm, norm_type=2)
    optimizer.step()

In [ ]:
def begin(ver):
    start = time.clock()
    for epoch in range(start_epoch, start_epoch+params.total_epoch):
        try:
            with tqdm(range(400)) as t:
                for step in t:
                    train()
                    if epoch>2 and (step+1)%100==0:
                        eval(epoch)
        except KeyboardInterrupt:
            t.close()
            raise
        state = {'net': model.state_dict(), 'optimizer':optimizer.state_dict(), 'epoch':epoch}
        torch.save(state, 'Model/treelstm'+ver+'.pth')
    end = time.clock()
    print("training time: ", end-start, " s")

# Begin

In [ ]:
# default
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 1
params.total_epoch = 6
params.max_norm = 10

## dropout

In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.0 ##
params.alpha = 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_p00')

 25%|██▌       | 100/400 [02:25<1:17:58, 15.60s/it]

epoch: 1, accuracy:  28.33%, loss_dect: 2.7800, loss_dep: 2.7713, loss_total: 5.5514, f1_macro: 14.61%


 50%|█████     | 200/400 [04:50<52:41, 15.81s/it]

epoch: 1, accuracy:  52.56%, loss_dect: 1.9183, loss_dep: 1.9104, loss_total: 3.8287, f1_macro: 48.99%


 75%|███████▌  | 300/400 [07:19<26:34, 15.95s/it]

...

epoch: 5, accuracy:  73.00%, loss_dect: 1.1329, loss_dep: 1.1432, loss_total: 2.2761, f1_macro: 74.25%


 75%|███████▌  | 300/400 [07:12<26:58, 16.19s/it]

epoch: 5, accuracy:  70.93%, loss_dect: 1.2763, loss_dep: 1.2943, loss_total: 2.5706, f1_macro: 72.23%


100%|██████████| 400/400 [09:41<00:00,  1.45s/it]

epoch: 5, accuracy:  68.07%, loss_dect: 1.3887, loss_dep: 1.4094, loss_total: 2.7980, f1_macro: 69.40%
training time:  2941.8748619999997  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.1 ##
params.alpha = 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_p01')

 25%|██▌       | 100/400 [02:27<1:18:09, 15.63s/it]

epoch: 1, accuracy:  31.07%, loss_dect: 2.7719, loss_dep: 2.8002, loss_total: 5.5721, f1_macro: 17.99%


 50%|█████     | 200/400 [04:50<50:57, 15.29s/it]

epoch: 1, accuracy:  54.15%, loss_dect: 1.8125, loss_dep: 1.8334, loss_total: 3.6459, f1_macro: 52.74%


 75%|███████▌  | 300/400 [07:11<25:19, 15.19s/it]

...

epoch: 5, accuracy:  70.78%, loss_dect: 1.1867, loss_dep: 1.1948, loss_total: 2.3815, f1_macro: 72.85%


 75%|███████▌  | 300/400 [07:10<26:03, 15.64s/it]

epoch: 5, accuracy:  70.59%, loss_dect: 1.2512, loss_dep: 1.2611, loss_total: 2.5123, f1_macro: 72.19%


100%|██████████| 400/400 [09:32<00:00,  1.43s/it]

epoch: 5, accuracy:  66.70%, loss_dect: 1.5992, loss_dep: 1.5950, loss_total: 3.1942, f1_macro: 67.41%
training time:  2851.4188779999986  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.2 ## 0.3
params.alpha = 1
params.total_epoch = 6
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_p02')

 25%|██▌       | 100/400 [02:22<1:16:23, 15.28s/it]

epoch: 1, accuracy:  36.56%, loss_dect: 2.7099, loss_dep: 2.7391, loss_total: 5.4489, f1_macro: 25.43%


 50%|█████     | 200/400 [04:44<51:25, 15.43s/it]

epoch: 1, accuracy:  51.00%, loss_dect: 1.8637, loss_dep: 1.8892, loss_total: 3.7529, f1_macro: 49.38%


 75%|███████▌  | 300/400 [07:09<26:09, 15.69s/it]

...

epoch: 6, accuracy:  70.74%, loss_dect: 1.3510, loss_dep: 1.3678, loss_total: 2.7188, f1_macro: 72.42%


 75%|███████▌  | 300/400 [07:21<26:24, 15.84s/it]

epoch: 6, accuracy:  70.33%, loss_dect: 1.3826, loss_dep: 1.3712, loss_total: 2.7539, f1_macro: 72.01%


100%|██████████| 400/400 [09:48<00:00,  1.47s/it]

epoch: 6, accuracy:  70.52%, loss_dect: 1.3638, loss_dep: 1.3856, loss_total: 2.7493, f1_macro: 71.27%
training time:  3556.619482  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3 ## 0.3
params.alpha = 1
params.total_epoch = 6
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_p03')

 25%|██▌       | 100/400 [02:27<1:19:01, 15.81s/it]

epoch: 1, accuracy:  31.81%, loss_dect: 2.7439, loss_dep: 2.7697, loss_total: 5.5137, f1_macro: 18.65%


 50%|█████     | 200/400 [04:54<52:51, 15.86s/it]

epoch: 1, accuracy:  51.85%, loss_dect: 1.8573, loss_dep: 1.8778, loss_total: 3.7352, f1_macro: 49.65%


 75%|███████▌  | 300/400 [07:26<28:12, 16.93s/it]

...

epoch: 6, accuracy:  72.00%, loss_dect: 1.1994, loss_dep: 1.2158, loss_total: 2.4152, f1_macro: 74.40%


 75%|███████▌  | 300/400 [07:30<26:57, 16.17s/it]

epoch: 6, accuracy:  72.15%, loss_dect: 1.3054, loss_dep: 1.2715, loss_total: 2.5770, f1_macro: 74.16%


100%|██████████| 400/400 [10:00<00:00,  1.50s/it]

epoch: 6, accuracy:  70.78%, loss_dect: 1.2972, loss_dep: 1.3131, loss_total: 2.6103, f1_macro: 73.27%
training time:  3624.424258000001  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.4 ## 0.3
params.alpha = 1
params.total_epoch = 6
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_p04')

 25%|██▌       | 100/400 [02:30<1:20:46, 16.15s/it]

epoch: 1, accuracy:  30.44%, loss_dect: 2.6205, loss_dep: 2.6470, loss_total: 5.2675, f1_macro: 16.14%


 50%|█████     | 200/400 [04:59<54:05, 16.23s/it]

epoch: 1, accuracy:  55.00%, loss_dect: 1.7930, loss_dep: 1.7831, loss_total: 3.5760, f1_macro: 54.28%


 75%|███████▌  | 300/400 [07:30<27:08, 16.29s/it]

...

epoch: 6, accuracy:  72.41%, loss_dect: 1.2116, loss_dep: 1.2336, loss_total: 2.4453, f1_macro: 74.54%


 75%|███████▌  | 300/400 [07:35<27:29, 16.49s/it]

epoch: 6, accuracy:  70.33%, loss_dect: 1.2888, loss_dep: 1.3131, loss_total: 2.6020, f1_macro: 71.66%


100%|██████████| 400/400 [10:06<00:00,  1.52s/it]

epoch: 6, accuracy:  68.44%, loss_dect: 1.4745, loss_dep: 1.4705, loss_total: 2.9450, f1_macro: 71.14%
training time:  3695.366894999999  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.5 ## 0.3
params.alpha = 1
params.total_epoch = 6
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_p05')

 25%|██▌       | 100/400 [02:33<1:22:51, 16.57s/it]

epoch: 1, accuracy:  30.93%, loss_dect: 2.7029, loss_dep: 2.7314, loss_total: 5.4343, f1_macro: 17.06%


 50%|█████     | 200/400 [05:05<55:08, 16.54s/it]

epoch: 1, accuracy:  51.74%, loss_dect: 1.8595, loss_dep: 1.8804, loss_total: 3.7399, f1_macro: 49.78%


 75%|███████▌  | 300/400 [07:38<27:58, 16.79s/it]

...

epoch: 6, accuracy:  70.93%, loss_dect: 1.2498, loss_dep: 1.2680, loss_total: 2.5178, f1_macro: 72.49%


 75%|███████▌  | 300/400 [07:39<27:33, 16.53s/it]

epoch: 6, accuracy:  71.15%, loss_dect: 1.2410, loss_dep: 1.2646, loss_total: 2.5056, f1_macro: 72.68%


100%|██████████| 400/400 [10:11<00:00,  1.53s/it]

epoch: 6, accuracy:  68.48%, loss_dect: 1.3821, loss_dep: 1.4078, loss_total: 2.7899, f1_macro: 69.49%
training time:  3710.949249000001  s


## alpha

In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 0 ## 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_p00') # it should be _a00

 25%|██▌       | 100/400 [02:35<1:23:11, 16.64s/it]

epoch: 1, accuracy:  52.85%, loss_dect: 1.9352, loss_dep: 1.9120, loss_total: 3.8472, f1_macro: 49.43%


 50%|█████     | 200/400 [05:05<53:32, 16.06s/it]

epoch: 1, accuracy:  64.15%, loss_dect: 1.3567, loss_dep: 1.3219, loss_total: 2.6786, f1_macro: 64.88%


 75%|███████▌  | 300/400 [07:31<26:13, 15.74s/it]

...

epoch: 5, accuracy:  73.33%, loss_dect: 1.3291, loss_dep: 1.2922, loss_total: 2.6213, f1_macro: 75.74%


 75%|███████▌  | 300/400 [07:07<25:27, 15.28s/it]

epoch: 5, accuracy:  74.15%, loss_dect: 1.3275, loss_dep: 1.3016, loss_total: 2.6291, f1_macro: 77.47%


100%|██████████| 400/400 [09:30<00:00,  1.43s/it]

epoch: 5, accuracy:  70.37%, loss_dect: 1.5491, loss_dep: 1.5246, loss_total: 3.0737, f1_macro: 73.34%
training time:  2911.1812610000015  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 0.1 ## 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_a01')

 25%|██▌       | 100/400 [02:22<1:16:47, 15.36s/it]

epoch: 1, accuracy:  39.93%, loss_dect: 2.2612, loss_dep: 2.2846, loss_total: 4.5458, f1_macro: 29.49%


 50%|█████     | 200/400 [04:44<51:17, 15.39s/it]

epoch: 1, accuracy:  59.67%, loss_dect: 1.5547, loss_dep: 1.5713, loss_total: 3.1261, f1_macro: 60.62%


 75%|███████▌  | 300/400 [07:07<25:39, 15.39s/it]

...

epoch: 5, accuracy:  73.52%, loss_dect: 1.1951, loss_dep: 1.2018, loss_total: 2.3969, f1_macro: 75.66%


 75%|███████▌  | 300/400 [07:07<25:41, 15.41s/it]

epoch: 5, accuracy:  72.52%, loss_dect: 1.2596, loss_dep: 1.2711, loss_total: 2.5308, f1_macro: 75.31%


100%|██████████| 400/400 [09:30<00:00,  1.43s/it]

epoch: 5, accuracy:  72.26%, loss_dect: 1.3262, loss_dep: 1.3222, loss_total: 2.6483, f1_macro: 74.20%
training time:  2875.316291000003  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 0.5 ## 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_a05')

 25%|██▌       | 100/400 [02:22<1:16:38, 15.33s/it]

epoch: 1, accuracy:  39.30%, loss_dect: 2.5518, loss_dep: 2.5764, loss_total: 5.1282, f1_macro: 25.68%


 50%|█████     | 200/400 [04:43<51:11, 15.36s/it]

epoch: 1, accuracy:  53.59%, loss_dect: 1.7560, loss_dep: 1.7781, loss_total: 3.5341, f1_macro: 52.93%


 75%|███████▌  | 300/400 [07:05<25:30, 15.30s/it]

...

epoch: 5, accuracy:  73.74%, loss_dect: 1.1315, loss_dep: 1.1410, loss_total: 2.2724, f1_macro: 74.89%


 75%|███████▌  | 300/400 [07:05<25:34, 15.35s/it]

epoch: 5, accuracy:  73.44%, loss_dect: 1.1346, loss_dep: 1.1360, loss_total: 2.2707, f1_macro: 75.34%


100%|██████████| 400/400 [09:27<00:00,  1.42s/it]

epoch: 5, accuracy:  71.63%, loss_dect: 1.2264, loss_dep: 1.2355, loss_total: 2.4619, f1_macro: 73.08%
training time:  2867.0038550000027  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 1 ## 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_a10')

 25%|██▌       | 100/400 [02:21<1:16:33, 15.31s/it]

epoch: 1, accuracy:  37.41%, loss_dect: 2.5803, loss_dep: 2.5655, loss_total: 5.1458, f1_macro: 27.63%


 50%|█████     | 200/400 [04:42<50:39, 15.20s/it]

epoch: 1, accuracy:  55.19%, loss_dect: 1.7986, loss_dep: 1.8154, loss_total: 3.6141, f1_macro: 53.44%


 75%|███████▌  | 300/400 [07:04<25:31, 15.32s/it]

...

epoch: 5, accuracy:  71.67%, loss_dect: 1.1517, loss_dep: 1.1428, loss_total: 2.2945, f1_macro: 75.39%


 75%|███████▌  | 300/400 [07:06<25:41, 15.41s/it]

epoch: 5, accuracy:  70.81%, loss_dect: 1.1693, loss_dep: 1.1799, loss_total: 2.3493, f1_macro: 73.61%


100%|██████████| 400/400 [09:28<00:00,  1.42s/it]

epoch: 5, accuracy:  70.00%, loss_dect: 1.2893, loss_dep: 1.3028, loss_total: 2.5921, f1_macro: 71.92%
training time:  2864.4567289999977  s


In [ ]:
train_data = prepare_input('data/train_path')
test_data = prepare_input('data/test_path')

In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-3 ## 1e-5
params.dropout = 0.3
params.alpha = 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_wd3')

 25%|██▌       | 100/400 [02:22<1:16:27, 15.29s/it]

epoch: 1, accuracy:  31.52%, loss_dect: 2.6679, loss_dep: 2.6604, loss_total: 5.3283, f1_macro: 18.68%


 50%|█████     | 200/400 [04:44<51:09, 15.35s/it]

epoch: 1, accuracy:  52.41%, loss_dect: 1.8869, loss_dep: 1.8873, loss_total: 3.7742, f1_macro: 49.06%


 75%|███████▌  | 300/400 [07:06<25:35, 15.35s/it]

...

epoch: 5, accuracy:  71.67%, loss_dect: 1.1391, loss_dep: 1.1482, loss_total: 2.2873, f1_macro: 74.50%


 75%|███████▌  | 300/400 [07:07<25:28, 15.29s/it]

epoch: 5, accuracy:  70.33%, loss_dect: 1.2764, loss_dep: 1.2919, loss_total: 2.5683, f1_macro: 71.79%


100%|██████████| 400/400 [09:29<00:00,  1.42s/it]

epoch: 5, accuracy:  67.67%, loss_dect: 1.4052, loss_dep: 1.4200, loss_total: 2.8252, f1_macro: 68.91%
training time:  2872.6120779999983  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-4 ## 1e-5
params.dropout = 0.3
params.alpha = 1
params.total_epoch = 4
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_wd4')

 25%|██▌       | 100/400 [02:22<1:16:27, 15.29s/it]

epoch: 1, accuracy:  32.15%, loss_dect: 2.7020, loss_dep: 2.7306, loss_total: 5.4325, f1_macro: 18.99%


 50%|█████     | 200/400 [04:44<50:53, 15.27s/it]

epoch: 1, accuracy:  50.52%, loss_dect: 1.8680, loss_dep: 1.8889, loss_total: 3.7569, f1_macro: 50.37%


 75%|███████▌  | 300/400 [07:07<25:39, 15.39s/it]

...

epoch: 5, accuracy:  71.37%, loss_dect: 1.2087, loss_dep: 1.2174, loss_total: 2.4261, f1_macro: 74.13%


 75%|███████▌  | 300/400 [07:07<25:27, 15.27s/it]

epoch: 5, accuracy:  70.19%, loss_dect: 1.2254, loss_dep: 1.2365, loss_total: 2.4618, f1_macro: 73.20%


100%|██████████| 400/400 [09:29<00:00,  1.42s/it]

epoch: 5, accuracy:  66.41%, loss_dect: 1.5127, loss_dep: 1.5066, loss_total: 3.0193, f1_macro: 69.25%
training time:  2873.9719200000036  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5 ## 1e-5
params.dropout = 0.3
params.alpha = 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_wd5')

 25%|██▌       | 100/400 [02:22<1:16:19, 15.27s/it]

epoch: 1, accuracy:  35.26%, loss_dect: 2.7187, loss_dep: 2.7469, loss_total: 5.4656, f1_macro: 24.67%


 50%|█████     | 200/400 [04:44<51:15, 15.38s/it]

epoch: 1, accuracy:  53.07%, loss_dect: 1.8943, loss_dep: 1.9182, loss_total: 3.8125, f1_macro: 52.59%


 75%|███████▌  | 300/400 [07:07<25:40, 15.40s/it]

...

epoch: 5, accuracy:  72.04%, loss_dect: 1.1319, loss_dep: 1.1388, loss_total: 2.2707, f1_macro: 73.54%


 75%|███████▌  | 300/400 [07:07<25:42, 15.42s/it]

epoch: 5, accuracy:  71.48%, loss_dect: 1.2113, loss_dep: 1.2099, loss_total: 2.4212, f1_macro: 73.53%


100%|██████████| 400/400 [09:30<00:00,  1.43s/it]

epoch: 5, accuracy:  70.85%, loss_dect: 1.3245, loss_dep: 1.3384, loss_total: 2.6628, f1_macro: 72.14%
training time:  2880.5008420000013  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-6 ## 1e-5
params.dropout = 0.3
params.alpha = 1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_wd6')

 25%|██▌       | 100/400 [02:22<1:16:31, 15.31s/it]

epoch: 1, accuracy:  31.15%, loss_dect: 2.7891, loss_dep: 2.7703, loss_total: 5.5594, f1_macro: 17.19%


 50%|█████     | 200/400 [04:43<51:15, 15.38s/it]

epoch: 1, accuracy:  54.56%, loss_dect: 1.7572, loss_dep: 1.7741, loss_total: 3.5313, f1_macro: 51.13%


 75%|███████▌  | 300/400 [07:06<25:34, 15.35s/it]

...

epoch: 5, accuracy:  72.22%, loss_dect: 1.1379, loss_dep: 1.1118, loss_total: 2.2496, f1_macro: 74.76%


 75%|███████▌  | 300/400 [07:06<25:31, 15.32s/it]

epoch: 5, accuracy:  71.93%, loss_dect: 1.1296, loss_dep: 1.1397, loss_total: 2.2693, f1_macro: 73.94%


100%|██████████| 400/400 [09:29<00:00,  1.42s/it]

epoch: 5, accuracy:  69.78%, loss_dect: 1.2589, loss_dep: 1.2690, loss_total: 2.5278, f1_macro: 72.24%
training time:  2876.4146379999947  s


## cnn

In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 0.1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_cnn')

 25%|██▌       | 100/400 [02:16<1:13:33, 14.71s/it]

epoch: 3, accuracy:  70.63%, loss_dect: 1.1551, loss_dep: 1.1571, loss_total: 2.3122, R: 72.95%, P: 75.06%, f1_macro: 73.25%


 50%|█████     | 200/400 [04:33<49:22, 14.81s/it]

epoch: 3, accuracy:  70.33%, loss_dect: 1.2380, loss_dep: 1.2386, loss_total: 2.4766, R: 70.39%, P: 77.02%, f1_macro: 72.87%


 75%|███████▌  | 300/400 [06:51<24:31, 14.71s/it]

...

epoch: 5, accuracy:  69.07%, loss_dect: 1.5380, loss_dep: 1.5585, loss_total: 3.0966, R: 68.38%, P: 75.68%, f1_macro: 71.55%


 75%|███████▌  | 300/400 [06:51<24:39, 14.80s/it]

epoch: 5, accuracy:  70.56%, loss_dect: 1.5219, loss_dep: 1.5434, loss_total: 3.0653, R: 70.62%, P: 75.40%, f1_macro: 72.26%


100%|██████████| 400/400 [09:08<00:00,  1.37s/it]

epoch: 5, accuracy:  64.37%, loss_dect: 1.9417, loss_dep: 1.9608, loss_total: 3.9025, R: 61.43%, P: 76.07%, f1_macro: 65.63%
training time:  2408.2018250000037  s


In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 0.1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_cnn2')

 25%|██▌       | 100/400 [02:18<1:14:26, 14.89s/it]

epoch: 3, accuracy:  68.33%, loss_dect: 1.2038, loss_dep: 1.2080, loss_total: 2.4117, R: 70.93%, P: 72.16%, f1_macro: 70.69%


 50%|█████     | 200/400 [04:36<49:18, 14.79s/it]

epoch: 3, accuracy:  68.67%, loss_dect: 1.2887, loss_dep: 1.2979, loss_total: 2.5866, R: 70.18%, P: 73.07%, f1_macro: 71.07%


 75%|███████▌  | 300/400 [06:55<24:50, 14.91s/it]

...

epoch: 5, accuracy:  68.44%, loss_dect: 1.5101, loss_dep: 1.5287, loss_total: 3.0388, R: 67.73%, P: 74.63%, f1_macro: 70.52%


 75%|███████▌  | 300/400 [06:53<24:30, 14.70s/it]

epoch: 5, accuracy:  67.30%, loss_dect: 1.5528, loss_dep: 1.5730, loss_total: 3.1258, R: 68.19%, P: 70.64%, f1_macro: 67.57%


100%|██████████| 400/400 [09:11<00:00,  1.38s/it]

epoch: 5, accuracy:  59.41%, loss_dect: 2.1654, loss_dep: 2.1814, loss_total: 4.3468, R: 54.63%, P: 72.51%, f1_macro: 58.35%
training time:  2420.511019999998  s


## SP-Tree

In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 0.1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_sp')

 25%|██▌       | 100/400 [01:41<58:04, 11.61s/it]

epoch: 3, accuracy:  74.11%, loss_dect: 0.9820, loss_dep: 0.9782, loss_total: 1.9601, R: 78.44%, P: 77.61%, f1_macro: 77.57%


 50%|█████     | 200/400 [03:22<38:23, 11.52s/it]

epoch: 3, accuracy:  75.07%, loss_dect: 0.9804, loss_dep: 0.9841, loss_total: 1.9645, R: 77.46%, P: 79.50%, f1_macro: 78.16%


 75%|███████▌  | 300/400 [05:04<19:19, 11.60s/it]

...

epoch: 5, accuracy:  74.37%, loss_dect: 1.1549, loss_dep: 1.1688, loss_total: 2.3237, R: 74.93%, P: 79.59%, f1_macro: 76.85%


 75%|███████▌  | 300/400 [05:03<19:07, 11.48s/it]

epoch: 5, accuracy:  72.74%, loss_dect: 1.2139, loss_dep: 1.2325, loss_total: 2.4465, R: 76.13%, P: 77.53%, f1_macro: 75.93%


100%|██████████| 400/400 [06:45<00:00,  1.01s/it]

epoch: 5, accuracy:  72.81%, loss_dect: 1.3114, loss_dep: 1.3268, loss_total: 2.6382, R: 72.34%, P: 78.49%, f1_macro: 74.67%
training time:  1778.8898530000006  s


## Baseline

In [ ]:
params.learn_rate = 1e-3
params.learn_rate_decay = 0.92
params.step_size = 50
params.weight_decay = 1e-5
params.dropout = 0.3
params.alpha = 0.1
params.total_epoch = 5
params.max_norm = 10

model = BiLSTM_TreeLSTM()
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=params.learn_rate, weight_decay=params.weight_decay, amsgrad=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, params.step_size, params.learn_rate_decay)
start_epoch = 1
begin('_base')

 25%|██▌       | 100/400 [02:23<1:17:52, 15.58s/it]

epoch: 3, accuracy:  73.81%, loss_dect: 1.0226, loss_dep: 1.0243, loss_total: 2.0468, R: 77.29%, P: 76.38%, f1_macro: 76.31%


 50%|█████     | 200/400 [04:46<51:37, 15.49s/it]

epoch: 3, accuracy:  74.78%, loss_dect: 0.9950, loss_dep: 1.0029, loss_total: 1.9980, R: 76.56%, P: 79.63%, f1_macro: 77.56%


 75%|███████▌  | 300/400 [07:10<25:50, 15.50s/it]

...

epoch: 5, accuracy:  74.93%, loss_dect: 1.1412, loss_dep: 1.1545, loss_total: 2.2956, R: 76.21%, P: 79.78%, f1_macro: 77.45%


 75%|███████▌  | 300/400 [07:06<25:32, 15.33s/it]

epoch: 5, accuracy:  73.44%, loss_dect: 1.2181, loss_dep: 1.2340, loss_total: 2.4522, R: 76.90%, P: 75.69%, f1_macro: 75.52%


100%|██████████| 400/400 [09:28<00:00,  1.42s/it]

epoch: 5, accuracy:  71.15%, loss_dect: 1.3384, loss_dep: 1.3514, loss_total: 2.6898, R: 71.62%, P: 77.07%, f1_macro: 72.59%
training time:  2517.7467939999988  s
